In [1]:
import numpy as np
import pandas as pd

In [3]:
# replace with your CSV
df = pd.read_csv("Dataframe_RBP_PDB.csv")

C:\Users\faris\AppData\Local\Temp\ipykernel_46960\143488485.py:1: DtypeWarning: Columns (11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  rbps_with_pdbs = pd.read_csv("Dataframe_RBP_PDB.csv")


In [6]:
rbds = pd.read_csv("1-s2.csv")

In [8]:
rbds_filtered = rbds[rbds["ProtID"].notna()]

In [10]:
grouped_rbds = rbds_filtered.groupby("ProtID")

In [14]:
dupes = rbds_filtered.duplicated(
    subset=["MS-identified tryptic peptide (N-link)", "LysC/ArgC proteolytic fragment (RBDpep)"],
    keep=False
)

,ENSG,ProtID,Symbol,MS-identified tryptic peptide (N-link),Start,Stop,category,Uniqueness,domain,enzyme,LysC/ArgC proteolytic fragment (RBDpep),fragmentStart,fragmentStop


In [15]:
def get_sequence_code(uniprot_id, sequence, rbds_subset):
    codes = []

    # Filter RBD entries for this UniProt ID
    matches = rbds_subset[rbds_subset["ProtID"] == uniprot_id]

    for _, row in matches.iterrows():
        rbd = row["LysC/ArgC proteolytic fragment (RBDpep)"]
        n_link = row["MS-identified tryptic peptide (N-link)"]
        domain = row["domain"]

        # Check if RBDpep is in sequence
        if rbd not in sequence:
            codes.append(0)
            continue

        # RBD is present — now check for N-link
        has_nlink = n_link in sequence

        # Assign based on domain and N-link presence
        if domain == "classical":
            codes.append(1 if has_nlink else 2)
        elif domain == "non-classical":
            codes.append(3 if has_nlink else 4)
        elif domain == "other":
            codes.append(5 if has_nlink else 6)
        else:
            codes.append(0)  # fallback for unexpected domain

    return codes if codes else [0]

In [16]:
df["Sequence_Code"] = df.apply(
    lambda row: get_sequence_code(row["uniprot_id"], row["sequence"], rbds_filtered),
    axis=1
)

In [25]:
def code_sequence(uniprot_id, sequence, rbds_subset):
    if not isinstance(sequence, str):
        return ""  # or return '0' * expected_length if you want placeholders
    code_array = np.zeros(len(sequence), dtype=int)  # Default all positions to 0
    matches = rbds_subset[rbds_subset["ProtID"] == uniprot_id]

    for _, row in matches.iterrows():
        rbdpep = row["LysC/ArgC proteolytic fragment (RBDpep)"]
        n_link = row["MS-identified tryptic peptide (N-link)"]
        domain = row["domain"]

        # Verify RBDpep is in sequence
        if rbdpep not in sequence:
            continue  # skip if it's not valid

        # Locate positions
        rbd_start = sequence.find(rbdpep)
        rbd_end = rbd_start + len(rbdpep)

        # Check if N-link is within RBDpep and inside sequence
        has_nlink = n_link in sequence
        n_start = sequence.find(n_link) if has_nlink else -1
        n_end = n_start + len(n_link) if has_nlink else -1

        # Determine code base for domain
        base = {"classical": 1, "non-classical": 3, "other": 5}.get(domain, 0)

        # Assign RBDpep region
        code_array[rbd_start:rbd_end] = base + 1  # RBD-only

        # If N-link present, overwrite with base (full match)
        if has_nlink:
            code_array[n_start:n_end] = base  # Full match

    return ''.join(code_array.astype(str))

In [26]:
df["Sequence_Code"] = df.apply(
    lambda row: code_sequence(row["uniprot_id"], row["sequence"], rbds_filtered),
    axis=1
)

In [22]:
rbds_filtered = rbds_filtered[
    rbds_filtered["MS-identified tryptic peptide (N-link)"].notna() &
    rbds_filtered["LysC/ArgC proteolytic fragment (RBDpep)"].notna()
]

In [24]:
rbds_filtered

,ENSG,ProtID,Symbol,MS-identified tryptic peptide (N-link),Start,Stop,category,Uniqueness,domain,enzyme,LysC/ArgC proteolytic fragment (RBDpep),fragmentStart,fragmentStop
0,ENSG00000138758,Q9NVA2,SEPT11,AAAQLLQSQAQQSGAQQTK,400.0,418.0,RBDpep,UniqueGene,other,LysC,KKAAAQLLQSQAQQSGAQQTK,398.0,418.0
1,ENSG00000116001,P31483,TIA1,AAFAPFGR,124.0,131.0,RBDpep,UniqueGene,classical,LysC,AAFAPFGRISDARVVK,124.0,139.0
2,ENSG00000007080,Q96CT7,CCDC124,AAFTAFEEAQLPR,171.0,183.0,RBDpep,UniqueGene,other,LysC,AKSHLEVPLEENVNRRVLEEGSVEARTIEDAIAVLSVAEEAADRHP...,120.0,185.0
3,ENSG00000149187,G5EA30,CELF1,AALEAQNALHNMK,98.0,110.0,RBDpep,UniqueGene,classical,LysC,AALEAQNALHNMK,98.0,110.0
4,ENSG00000145920,Q6PUV4,CPLX2,AALEQPCEGSLTRPK,84.0,98.0,RBDpep,UniqueGene,other,LysC,AALEQPCEGSLTRPK,84.0,98.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1375,ENSG00000059573,P54886,ALDH18A1,YLHENLPIPQR,782.0,792.0,RBDpep,UniqueGene,other,ArgC,GKDHVVSDFSEHGSLKYLHENLPIPQR,766.0,792.0
1376,ENSG00000187955,Q05707,COL14A1,YNVISHDSIQISWK,38.0,51.0,RBDpep,UniqueGene,other,ArgC,LRYNVISHDSIQISWKAPR,36.0,54.0
1377,ENSG00000189060,P07305,H1F0,YSDMIVAAIQAEK,28.0,40.0,RBDpep,UniqueGene,other,ArgC,AKASKKSTDHPKYSDMIVAAIQAEKNR,16.0,42.0
1378,ENSG00000265681,P18621,RPL17,YSLDPENPTK,4.0,13.0,RBDpep,UniqueGene,other,ArgC,MVRYSLDPENPTKSCKSR,1.0,18.0


In [27]:
rbps_with_pdbs

,UNIQUE,ID,Name,Description,HEK293-RIC,HuH7-IC,HeLa-RNPxl,HeLa-IC,HeLa-RBDmap,K562-serIC-chr,...,Number of Large ribosomal subunit protein uL23 N-terminal,Average Size of Large ribosomal subunit protein uL23 N-terminal,Total Size of Large ribosomal subunit protein uL23 N-terminal,Number of IMP dehydrogenase/GMP reductase,Average Size of IMP dehydrogenase/GMP reductase,Total Size of IMP dehydrogenase/GMP reductase,PDB_IDs,PDB_Type,PDB_Paths,Sequence_Code
0,A1CF,ENSG00000148584,A1CF,APOBEC1 complementation factor,no,YES,no,no,YES,no,...,0.0,0.0,0.0,0.0,0.0,0.0,['2CPD'],Experimental,['experimental_pdbs\\2CPD.cif'],0000000000000000000000000000000000000000000000...
1,AARS,ENSG00000090861,AARS,alanyl-tRNA synthetase,no,YES,no,no,no,no,...,0.0,0.0,0.0,0.0,0.0,0.0,"['4XEM', '4XEO', '5KNN', '5T5S', '5T76', '5V59']",Experimental,"['experimental_pdbs\\4XEM.cif', 'experimental_...",0000000000000000000000000000000000000000000000...
2,AATF,ENSG00000276072;ENSG00000275700,AATF,apoptosis antagonizing transcription factor,YES,no,no,no,no,no,...,0.0,0.0,0.0,0.0,0.0,0.0,"['5W6A', '7MQ8', '7MQ9']",Experimental,"['experimental_pdbs\\5W6A.cif', 'experimental_...",0000000000000000000000000000000000000000000000...
3,ABCE1,ENSG00000164163,ABCE1,ATP binding cassette subfamily E member 1,no,no,no,no,YES,no,...,0.0,0.0,0.0,0.0,0.0,0.0,"['6ZME', '6ZVJ', '7A09']",Experimental,"['experimental_pdbs\\6ZME.cif', 'experimental_...",0000000000000000000000000000000000000000000000...
4,ABCF1,ENSG00000225989;ENSG00000232169;ENSG0000020457...,ABCF1,ATP binding cassette subfamily F member 1,YES,no,no,YES,YES,no,...,0.0,0.0,0.0,0.0,0.0,0.0,['5ZXD'],Experimental,['experimental_pdbs\\5ZXD.cif'],0000000000000000000000000000000000000000000000...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1408,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,AlphaFold,['alphafold_pdbs\\J3KTL2.cif'],2222222222222222222222222222221111111122222000...
1409,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,AlphaFold,['alphafold_pdbs\\Q5T1Z8.cif'],0000000000000000000000000000000000000000000000...
1410,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,AlphaFold,['alphafold_pdbs\\Q5VWC4.cif'],0000000000000000000000000000000000000000000000...
1411,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,AlphaFold,['alphafold_pdbs\\Q6IPX4.cif'],6666555555555556000000000000000000000000000000...


In [29]:
rbps_with_pdbs["RBDpep_Code"][0]

'000000000000000000000000000000000000000000000000000000000000002221111111111111000000000000000000000000000000000000111111112222222222222222222222200000000000000000000000000000000000000000000000000000000000000000000000000000000000000222221111111111111000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'

In [30]:
rbps_with_pdbs.to_csv("RBP_RBD_PDB.csv", index=False)